# Azure Maps Render API - Static Map Image Generator

This notebook processes a list of site locations with GPS coordinates and generates static map images using the Azure Maps Render API.

## Features:
- Load site coordinates from a list
- Call Azure Maps Render API
- Save PNG images with customizable map styles
- Support for pushpins and custom labels

## 1. Import Required Libraries

In [ ]:
import os
import requests
from pathlib import Path
from typing import List, Dict
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime

## 2. Load Environment Variables

In [ ]:
# Load environment variables from .env file
env_path = Path.cwd().parent / '.env'
load_dotenv(dotenv_path=env_path)

# Get Azure Maps credentials
AZURE_MAPS_CLIENT_ID = os.getenv('AZURE_MAPS_CLIENT_ID')
AZURE_MAPS_SUBSCRIPTION_KEY = os.getenv('AZURE_MAPS_SUBSCRIPTION_KEY')

# Validate credentials
if not AZURE_MAPS_SUBSCRIPTION_KEY:
    raise ValueError("AZURE_MAPS_SUBSCRIPTION_KEY not found in .env file")

print("✓ Azure Maps credentials loaded successfully")
print(f"  Client ID: {'*' * 10 if AZURE_MAPS_CLIENT_ID else 'Not set (using subscription key)'}")
print(f"  Subscription Key: {'*' * 10}")

## 3. Configure Azure Maps API Settings

In [ ]:
# Azure Maps Render API configuration
API_BASE_URL = "https://atlas.microsoft.com/map/static"
API_VERSION = "2024-04-01"

# Default map settings
DEFAULT_CONFIG = {
    'tilesetId': 'microsoft.base.road',  # Options: microsoft.base.road, microsoft.base.darkgrey, microsoft.imagery
    'zoom': 12,                          # Range: 0-20
    'width': 800,                        # Range: 80-2000 pixels
    'height': 600,                       # Range: 80-1500 pixels
    'language': 'en-US',
    'view': 'Unified',
    'format': 'png'                      # Options: png, jpeg
}

# Output directory for saved images
OUTPUT_DIR = Path(r'c:\temp\azure_maps_images')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"✓ Configuration complete")
print(f"  Output directory: {OUTPUT_DIR}")

## 4. Define Site Locations

Create a list of sites with GPS coordinates and labels.

In [ ]:
# Define your site locations here
sites = [
    {
        'name': 'Microsoft HQ',
        'latitude': 47.6423,
        'longitude': -122.1390,
        'label': 'Microsoft Campus'
    },
    {
        'name': 'Space Needle',
        'latitude': 47.6205,
        'longitude': -122.3493,
        'label': 'Space Needle'
    },
    {
        'name': 'Pike Place Market',
        'latitude': 47.6097,
        'longitude': -122.3422,
        'label': 'Pike Place'
    },
    # Add more sites as needed
]

# Display sites
df_sites = pd.DataFrame(sites)
print(f"\n{len(sites)} sites configured:")
df_sites

## 5. Azure Maps API Functions

In [ ]:
def build_map_url(lon: float, lat: float, label: str = None, config: dict = None) -> str:
    """
    Build Azure Maps Static Image API URL with parameters.
    
    Args:
        lon: Longitude (-180 to 180)
        lat: Latitude (-90 to 90)
        label: Optional label for the pushpin
        config: Optional configuration dictionary to override defaults
    
    Returns:
        Complete API URL with query parameters
    """
    # Merge default config with custom config
    settings = {**DEFAULT_CONFIG, **(config or {})}
    
    # Build query parameters
    params = {
        'api-version': API_VERSION,
        'subscription-key': AZURE_MAPS_SUBSCRIPTION_KEY,
        'center': f"{lon},{lat}",
        'zoom': settings['zoom'],
        'width': settings['width'],
        'height': settings['height'],
        'tilesetId': settings['tilesetId'],
        'language': settings['language'],
        'view': settings['view']
    }
    
    # Add pushpin if label is provided
    if label:
        # Format: style||lon lat||label
        # Styles: default, small, large
        params['pins'] = f"default||{lon} {lat}||{label}"
    
    # Add client ID header if available (for AAD authentication)
    if AZURE_MAPS_CLIENT_ID:
        params['x-ms-client-id'] = AZURE_MAPS_CLIENT_ID
    
    return API_BASE_URL, params


def fetch_map_image(lon: float, lat: float, label: str = None, config: dict = None) -> bytes:
    """
    Fetch static map image from Azure Maps API.
    
    Args:
        lon: Longitude
        lat: Latitude
        label: Optional pushpin label
        config: Optional configuration overrides
    
    Returns:
        Image content as bytes
    """
    url, params = build_map_url(lon, lat, label, config)
    
    headers = {
        'Accept': 'image/png'
    }
    
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    
    return response.content


def save_map_image(image_data: bytes, filename: str) -> Path:
    """
    Save map image to file.
    
    Args:
        image_data: Image content as bytes
        filename: Name for the output file (without extension)
    
    Returns:
        Path to saved file
    """
    filepath = OUTPUT_DIR / f"{filename}.png"
    
    with open(filepath, 'wb') as f:
        f.write(image_data)
    
    return filepath


print("✓ API functions defined")

## 6. Generate Map Images for All Sites

In [ ]:
# Process each site
results = []

for site in sites:
    try:
        print(f"\nProcessing: {site['name']}...")
        
        # Fetch map image
        image_data = fetch_map_image(
            lon=site['longitude'],
            lat=site['latitude'],
            label=site.get('label', site['name'])
        )
        
        # Create safe filename
        safe_name = "".join(c if c.isalnum() or c in (' ', '-', '_') else '_' for c in site['name'])
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{safe_name}_{timestamp}"
        
        # Save image
        filepath = save_map_image(image_data, filename)
        
        results.append({
            'site_name': site['name'],
            'latitude': site['latitude'],
            'longitude': site['longitude'],
            'status': 'Success',
            'filepath': str(filepath),
            'file_size_kb': len(image_data) / 1024
        })
        
        print(f"  ✓ Saved to: {filepath}")
        print(f"  Size: {len(image_data) / 1024:.1f} KB")
        
    except Exception as e:
        print(f"  ✗ Error: {str(e)}")
        results.append({
            'site_name': site['name'],
            'latitude': site['latitude'],
            'longitude': site['longitude'],
            'status': 'Failed',
            'filepath': None,
            'error': str(e)
        })

# Display results
df_results = pd.DataFrame(results)
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"Total sites processed: {len(results)}")
print(f"Successful: {len([r for r in results if r['status'] == 'Success'])}")
print(f"Failed: {len([r for r in results if r['status'] == 'Failed'])}")
print(f"\nOutput directory: {OUTPUT_DIR}")
print("\nDetailed Results:")
df_results

## 7. Advanced: Generate Map with Custom Style

In [ ]:
# Example: Generate a satellite imagery map with higher zoom
custom_config = {
    'tilesetId': 'microsoft.imagery',  # Satellite imagery
    'zoom': 15,                        # Closer zoom
    'width': 1200,                     # Larger image
    'height': 900
}

# Pick a site to demonstrate
if sites:
    demo_site = sites[0]
    print(f"Generating satellite view for: {demo_site['name']}")
    
    image_data = fetch_map_image(
        lon=demo_site['longitude'],
        lat=demo_site['latitude'],
        label=demo_site.get('label'),
        config=custom_config
    )
    
    filepath = save_map_image(image_data, f"{demo_site['name']}_satellite")
    print(f"✓ Satellite view saved to: {filepath}")
    print(f"  Size: {len(image_data) / 1024:.1f} KB")

## 8. Optional: Load Sites from CSV File

Instead of hardcoding sites, you can load them from a CSV file.

In [ ]:
# Example CSV format:
# name,latitude,longitude,label
# "Location 1",47.6062,-122.3321,"Label 1"
# "Location 2",47.6205,-122.3493,"Label 2"

def load_sites_from_csv(csv_path: str) -> List[Dict]:
    """
    Load site locations from CSV file.
    
    Expected columns: name, latitude, longitude, label (optional)
    """
    df = pd.read_csv(csv_path)
    
    required_cols = ['name', 'latitude', 'longitude']
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"CSV must contain columns: {required_cols}")
    
    sites = []
    for _, row in df.iterrows():
        site = {
            'name': row['name'],
            'latitude': float(row['latitude']),
            'longitude': float(row['longitude']),
        }
        if 'label' in row and pd.notna(row['label']):
            site['label'] = row['label']
        
        sites.append(site)
    
    return sites

# Uncomment to use:
# csv_file = r"c:\temp\sites.csv"
# sites = load_sites_from_csv(csv_file)
# print(f"Loaded {len(sites)} sites from CSV")

print("✓ CSV loader function defined")